In [23]:
# importing all the required libraries
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Data Preprocessing

In [8]:
# Loading the movies.csv dataset into the variable movies
movie=pd.read_csv('/content/data.csv')
# The data.csv file contains data of movies including the names of the cast, director, etc.

In [24]:
movie.head(10)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
5,5,258000000,Fantasy Action Adventure,http://www.sonypictures.com/movies/spider-man3/,559,dual identity amnesia sandstorm love of one's ...,en,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.699814,...,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The battle within.,Spider-Man 3,5.9,3576,Tobey Maguire Kirsten Dunst James Franco Thoma...,"[{'name': 'Francine Maisler', 'gender': 1, 'de...",Sam Raimi
6,6,260000000,Animation Family,http://disney.go.com/disneypictures/tangled/,38757,hostage magic horse fairy tale musical,en,Tangled,When the kingdom's most wanted-and most charmi...,48.681969,...,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,They're taking adventure to new lengths.,Tangled,7.4,3330,Zachary Levi Mandy Moore Donna Murphy Ron Perl...,"[{'name': 'John Lasseter', 'gender': 2, 'depar...",Byron Howard
7,7,280000000,Action Adventure Science Fiction,http://marvel.com/movies/movie/193/avengers_ag...,99861,marvel comic sequel superhero based on comic b...,en,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,134.279229,...,141.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Age Has Come.,Avengers: Age of Ultron,7.3,6767,Robert Downey Jr. Chris Hemsworth Mark Ruffalo...,"[{'name': 'Danny Elfman', 'gender': 2, 'depart...",Joss Whedon
8,8,250000000,Adventure Fantasy Family,http://harrypotter.warnerbros.com/harrypottera...,767,witch magic broom

In [25]:
# To view the names of the columns
movie.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [26]:
# to check the datatype of the columns
movie.dtypes

index                     int64
budget                    int64
genres                   object
homepage                 object
id                        int64
keywords                 object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
production_countries     object
release_date             object
revenue                   int64
runtime                 float64
spoken_languages         object
status                   object
tagline                  object
title                    object
vote_average            float64
vote_count                int64
cast                     object
crew                     object
director                 object
dtype: object

In [27]:
# to see the number of rows and columns 
movie.shape

(4803, 24)

In [28]:
# Selecting the features based on which we will be recommending movies 
selected_features=['genres','keywords','tagline','cast','director','original_language']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director', 'original_language']


In [29]:
movie.isnull().sum()

index                      0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                    0
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
director                   0
dtype: int64

In [30]:
# Now we will replace the null values with an empty string 
for feature in selected_features:
  movie[feature]=movie[feature].fillna('')

In [31]:
movie.isnull().sum()

index                      0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                    0
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
director                   0
dtype: int64

In [32]:
# So here we have replaced all the null values that existed in the selected columns with an empty string
# Now we will combine all the 6 selected features
combined_features= movie['genres']+''+movie['keywords']+''+movie['tagline']+''+movie['cast']+''+movie['director']+''+movie['original_language']

In [33]:
print(combined_features)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object


In [34]:
# we now convert the textual data to a feature vector 
# for this we will use the tfidfvectorizer function
# It converts text to numerical value 
# this is helpful for cosine similarity matrix generation
vectorizer=TfidfVectorizer()

In [35]:
feature_vector=vectorizer.fit_transform(combined_features)

In [36]:
feature_vector

<4803x28435 sparse matrix of type '<class 'numpy.float64'>'
	with 112149 stored elements in Compressed Sparse Row format>

In [37]:
print(feature_vector)

  (0, 3698)	0.2236154721374853
  (0, 21378)	0.2698154610688479
  (0, 17128)	0.15464383015586172
  (0, 14976)	0.23606993993734768
  (0, 23993)	0.15890071163256164
  (0, 27354)	0.19348783327407196
  (0, 23165)	0.19908811283673494
  (0, 22153)	0.21232025501103283
  (0, 28392)	0.19479511554510787
  (0, 28023)	0.2313349710341213
  (0, 22188)	0.1539294608176413
  (0, 19130)	0.25736099326898554
  (0, 18726)	0.08562545813720154
  (0, 28001)	0.1243360353680507
  (0, 24943)	0.07407562231265147
  (0, 23490)	0.2698154610688479
  (0, 4842)	0.24167021950001066
  (0, 27136)	0.12866947571628654
  (0, 23634)	0.3317176950110682
  (0, 10466)	0.16288396011055772
  (0, 4615)	0.21477888660570982
  (0, 9553)	0.2698154610688479
  (0, 22518)	0.09819475409717197
  (0, 9180)	0.11631860496928596
  (0, 449)	0.08974891372654038
  :	:
  (4801, 22857)	0.3149866898963259
  (4801, 28209)	0.3149866898963259
  (4801, 28365)	0.3004471539810804
  (4801, 8353)	0.2610521172477128
  (4801, 767)	0.19362615158831434
  (4801, 12

In [38]:
# Here we see that the textual data has been converted to numerical data
# We will now find the cosine similarity of the vectors generated
# for this we will use the cosine similarity function 
similarity=cosine_similarity(feature_vector)

In [39]:
print(similarity)

[[1.         0.06763832 0.01459506 ... 0.         0.         0.        ]
 [0.06763832 1.         0.02778726 ... 0.01237079 0.         0.        ]
 [0.01459506 0.02778726 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01237079 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [40]:
print(similarity.shape)

(4803, 4803)


In [41]:
# Here we see that a matrix has been generated with the similarity scores of all the combined features 


In [42]:
# Now we will take input from th user asking the user for a movie to provide recommendations
movie_name=input('Enter your favourite name : ')

Enter your favourite name : Spider Man


In [43]:
# we now generate a list of all the movies given in the dataset
list_of_names=movie['title'].tolist()

In [44]:
print(list_of_names)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [45]:
# Finding the closest match for the movie name given by the user
closest_match=difflib.get_close_matches(movie_name,list_of_names)
print(closest_match)

['Spider-Man', 'Spider-Man 3', 'Spider-Man 2']


In [46]:
close_match=closest_match[0]
print(close_match)

Spider-Man


In [47]:
movie_index=movie[movie.title == close_match]['index'].values[0]
print(movie_index)

159


In [48]:
# Now we will get a list of similar movies based on the index value obtined
similarity_score=list(enumerate(similarity[movie_index]))
print(similarity_score)

[(0, 0.019867310272197597), (1, 0.02382520477312184), (2, 0.0), (3, 0.0), (4, 0.06961873108039185), (5, 0.27903569321475136), (6, 0.0), (7, 0.0), (8, 0.021184273211493364), (9, 0.0), (10, 0.02989023099301592), (11, 0.0), (12, 0.01334105613288704), (13, 0.012636858461500294), (14, 0.0132895677044891), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.006153672092400538), (20, 0.006808934647227023), (21, 0.0), (22, 0.006700307929082661), (23, 0.0), (24, 0.006591454951207933), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.2658625871963497), (31, 0.028789285569757503), (32, 0.024044303389565664), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.0), (37, 0.09599080136882007), (38, 0.0), (39, 0.0), (40, 0.0), (41, 0.0), (42, 0.0), (43, 0.0), (44, 0.0), (45, 0.01223935022138727), (46, 0.024456783002551705), (47, 0.0), (48, 0.0), (49, 0.08128190660242475), (50, 0.013329178578733933), (51, 0.0), (52, 0.0), (53, 0.0), (54, 0.0), (55, 0.0), (56, 0.0), (57, 0.0), (58, 0.0), (59, 0.00686247

In [49]:
len(similarity_score)

4803

In [50]:

# In this the 1st value displays the index of the movie and the 2nd value displays the similarity score of the movie 
# now we will sort the movies based on their similarity score 

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)

[(159, 1.0000000000000002), (5, 0.27903569321475136), (30, 0.2658625871963497), (1559, 0.1579059886828584), (382, 0.15630477766978956), (1364, 0.1250758653651533), (328, 0.1184036309125841), (1523, 0.11212927684675524), (677, 0.11011007301203517), (1193, 0.10578377940226749), (2369, 0.09924206870973892), (1598, 0.09680526315498082), (37, 0.09599080136882007), (3046, 0.09350068688753435), (3188, 0.09171350409261184), (1427, 0.09107138644228313), (764, 0.08819676219104132), (2529, 0.08764823482637046), (2361, 0.08716013566526615), (3479, 0.08532034784860194), (1796, 0.08376286792744025), (868, 0.08374551212522285), (4441, 0.0832673413366564), (1435, 0.08313478108244637), (3130, 0.08161008703946615), (49, 0.08128190660242475), (3177, 0.08110990004745997), (912, 0.07949895085762416), (1191, 0.07908021045787843), (2375, 0.07846431541843323), (2310, 0.07754595058354141), (978, 0.07646261937067834), (525, 0.07584781796118034), (3445, 0.07549939506966262), (2979, 0.07528077254243597), (3846, 0

In [51]:
print('Movies suggested for you : \n')

i = 1

for mov in sorted_similar_movies:
  index = mov[0]
  title_from_index = movie[movie.index==index]['title'].values[0]
  if (i<=10):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Spider-Man
2 . Spider-Man 3
3 . Spider-Man 2
4 . The Notebook
5 . Seabiscuit
6 . Horrible Bosses
7 . Finding Nemo
8 . The Good German
9 . Clear and Present Danger
10 . The Count of Monte Cristo


In [155]:
# As we can see above we have recommended 10 movies to the user based on the movie name provided in the input box above 